In [1]:
from neo4j import GraphDatabase
from langchain.graphs import Neo4jGraph
from credentials import local_cred, remote_cred, openai_api_key
import pandas as pd
import os

In [2]:
# # URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
# URI = "neo4j+s://07c7bb01.databases.neo4j.io"
# AUTH = ("neo4j", "_JDmn-JKikceXc-XUeDLXaYMMKoFW_PX4MbZveGDbUU")

# with GraphDatabase.driver(URI, auth=AUTH) as driver:
#     driver.verify_connectivity()

In [19]:
# os.environ["NEO4J_URI"] = "neo4j+s://07c7bb01.databases.neo4j.io"
# os.environ["NEO4J_USERNAME"] = "neo4j"
# os.environ["NEO4J_PASSWORD"] =  "_JDmn-JKikceXc-XUeDLXaYMMKoFW_PX4MbZveGDbUU"

In [2]:
os.environ["NEO4J_URI"] = local_cred["uri"]
os.environ["NEO4J_USERNAME"] = local_cred["username"]
os.environ["NEO4J_PASSWORD"] =  local_cred["password"]

In [4]:
os.environ["OPENAI_API_KEY"] = openai_api_key

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv')

In [6]:
df

,movieId,released,title,actors,director,genres,imdbRating
0,1,1995-11-22,Toy Story,Jim Varney|Tim Allen|Tom Hanks|Don Rickles,John Lasseter,Adventure|Animation|Children|Comedy|Fantasy,8.3
1,2,1995-12-15,Jumanji,Robin Williams|Bradley Pierce|Kirsten Dunst|Jo...,Joe Johnston,Adventure|Children|Fantasy,6.9
2,3,1995-12-22,Grumpier Old Men,Walter Matthau|Ann-Margret|Jack Lemmon|Sophia ...,Howard Deutch,Comedy|Romance,6.6
3,4,1995-12-22,Waiting to Exhale,Whitney Houston|Lela Rochon|Angela Bassett|Lor...,Forest Whitaker,Romance|Drama|Comedy,5.6
4,5,1995-12-08,Father of the Bride Part II,Steve Martin|Kimberly Williams-Paisley|Diane K...,Charles Shyer,Comedy,5.9
...,...,...,...,...,...,...,...
294,329,1994-11-18,Star Trek: Generations,Patrick Stewart|Brent Spiner|Jonathan Frakes|L...,David Carson,Drama|Sci-Fi|Adventure,6.6
295,330,1995-05-24,Tales from the Hood,Samuel Monroe Jr.|De'aundre Bonds|Joe Torry|Cl...,Rusty Cundieff,Action|Crime|Horror,6.1
296,331,1994-12-02,Tom & Viv,Willem Dafoe|Miranda Richardson|Tim Dutton|Ros...,Brian Gilbert,Drama,6.5
297,332,1995-04-28,Village of the Damned,Christopher Reeve|Michael Paré|Kirstie Alley|L...,John Carpenter,Horror|Sci-Fi,5.6


In [4]:
graph = Neo4jGraph()

In [5]:
movies_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

graph.query(movies_query)

[]

In [12]:
# graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {imdbRating: FLOAT, id: STRING, released: DATE, title: STRING}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [10]:
graph._driver.close()

In [14]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

In [59]:
# llm = ChatOpenAI(model="gpt2-medium", temperature=0, openai_api_key=openai_api_key)
llm = ChatOllama(model='llama3', temperature=0)
chain = GraphCypherQAChain.from_llm(llm, graph=graph)

In [39]:
response = chain.invoke({"query": "What is the cast of Toy Story"})
print(response["result"])

The cast of Toy Story includes Jim Varney and Tim Allen.


In [68]:
response = chain.invoke({"query": "All the movies Christopher Nolan directed."})
print(response["result"])

I don't know the answer.


In [ ]:
response = chain.invoke({"query": "All the movies Tom Hanks acted in."})
print(response["result"])

In [28]:
response = chain.invoke({"query": "Give me the cast and director of Apollo 13. Also tell me all the details you know about the movie"})
print(response["result"])

The cast of Apollo 13 includes Tom Hanks, Bill Paxton, Kevin Bacon, Gary Sinise, and Ed Harris. The director is Ron Howard.

As for the details I know about the movie, Apollo 13 is a drama film that tells the story of the Apollo 13 mission, which was aborted after an explosion occurred on board the spacecraft. The movie follows the heroic efforts of astronauts Jim Lovell (Tom Hanks), Jack Swigert (Kevin Bacon), and Fred Haise (Bill Paxton) to safely return to Earth after the accident. The film also explores the role of Mission Control, led by Gene Kranz (Ed Harris), in guiding the astronauts through the crisis.


In [29]:
response = chain.invoke({"query": "Which are the top 5 movies according to your knowledge."})
print(response["result"])

The Shawshank Redemption, The and Pulp Fiction are the top 3 movies.


In [30]:
response = chain.invoke({"query": "Which is the worst movie?"})
print(response["result"])

The Lawnmower Man 2: Beyond Cyberspace.


In [60]:
response = chain.invoke({"query": "Which genre is most popular?"})
print(response["result"])

The genre that is most popular is Drama.


In [69]:
response = chain.invoke({"query": "Do you know about forest gump."})
print(response["result"])

I don't know the answer.


In [56]:
response

{'query': 'Write something funny if you do not know the answer.',
 'result': "I don't know the answer!"}